# Implementation of the `scaredy_fish` model in Julia
### First step
Let's set up blank arrays that we will fill in, draw initial conditions, and set parameter values

In [42]:
## ---
## init
## ---
using Distributions, Distances, IterTools, BenchmarkTools


## -
## Define my landscape
## -

## number of rows and cols
nrows = 10
ncols = 100

## landscape is a binary matrix, where 1s are foraging patches (white tiles)
landscape = rand( Bernoulli( 0.5 ), nrows, ncols )

## indices for white tiles
whiteTiles = getindex.( findall( y -> y == 1, landscape ), [ 1 2 ] )


## -
## Define a bunch of parameters
## -

## number of agents
agents = 100

## time steps
time = 10

## walk step, this is the SD of the normal distribution from which walk step lengths are drawn
walk = 0.05

## run step, this is the SD of the normal distribution from which run step lengths are drawn
run = 0.2

## strength of drift back toward white tile if you're in a black tile
drift = 0.2

## sensory length scale radius
sensory = 2.5

## trigger time for false startle
trigger = 5;

### Second step
Define a function that will loop through `times` and `agents` and do all the calculations

In [40]:
## ---
## Function to loop through time steps
## ---
function scaredyFish( )
    
    ## -
    ## Prep simulation, define some empty arrays to fill in
    ## -

    ## empty 3D array to populate ( time x number of variables x agents )
    r = zeros( time, 6, agents );

    ## populate first row of each matrix
    for agent in 1 : size( r, 3 )

        ## put a random number in the first row
        r[ 1, 1, agent ] = Float64.( rand( Uniform( 0.5, 0.5 + size( landscape, 1 ) ) ) )
        r[ 1, 2, agent ] = Float64.( rand( Uniform( 0.5, 0.5 + size( landscape, 2 ) ) ) )

    end
    
    ## -
    ## Start workhorse
    ## -
    
    ## for t in ___time___
    for t in 2 : time

        ## for a in agents
        for a in 1 : agents

            ## -
            ## first we need to know if we're in a white tile or black tile, and where the nearest white tile is
            ## -
            currX = r[ t - 1, 1, a ]
            currY = r[ t - 1, 2, a ]

            ## round them to find the nearest tile
            currPatchX = round.( Int, currX )
            currPatchY = round.( Int, currY )

            ## check that it is actually a tile, and you are not out of bounds
            if ( 0.5 < currPatchX < ( 0.5 + size( landscape, 1 ) ) ) && ( 0.5 < currPatchY < ( 0.5 + size( landscape, 2 ) ) )

                ## note that I am in bounds
                inBounds = 1

                ## if this is a white tile, we are already done
                if landscape[ currPatchX, currPatchY ] == 1

                    ## make this my home patch
                    homePatchX = currPatchX
                    homePatchY = currPatchY

                    ## record status in output
                    r[ t - 1, 4, a ] = 1
                    r[ t - 1, 5, a ] = homePatchX
                    r[ t - 1, 6, a ] = homePatchY

                end

                ## otherwise, find the nearest white tile
                if landscape[ currPatchX, currPatchY ] == 0

                    ## distances to white tiles
                    toWhiteTiles = colwise( Euclidean(), [currX, currY], transpose( whiteTiles ) )

                    ## which min
                    homePatchX = whiteTiles[ argmin( toWhiteTiles ), 1 ]
                    homePatchY = whiteTiles[ argmin( toWhiteTiles ), 2 ]

                    ## record status in output
                    r[ t - 1, 4, a ] = 0
                    r[ t - 1, 5, a ] = homePatchX
                    r[ t - 1, 6, a ] = homePatchY

                end

            ## if I'm out of bounds...
            else

                ## note it
                inBounds = 0

                ## distances to white tiles
                toWhiteTiles = colwise( Euclidean(), [currX, currY], transpose( whiteTiles ) )

                ## which min
                homePatchX = whiteTiles[ argmin( toWhiteTiles ), 1 ]
                homePatchY = whiteTiles[ argmin( toWhiteTiles ), 2 ]

                ## record status in output
                r[ t - 1, 4, a ] = 0
                r[ t - 1, 5, a ] = homePatchX
                r[ t - 1, 6, a ] = homePatchY

            end


            ## -
            ## Here starts the step updating section
            ## -

            ## if feeding and in bounds
            if r[ t - 1, 3, a ] == 0 && inBounds == 1

                ## random walk steps
                r[ t, 1, a ] = r[ t - 1, 1, a ] + rand( Normal( 0, walk ) )
                r[ t, 2, a ] = r[ t - 1, 2, a ] + rand( Normal( 0, walk ) )

            end

            ## if feeding and out of bounds
            if r[ t - 1, 3, a ] == 0 && inBounds == 0

                ## bias my walk in this direction
                xBias = ( homePatchX - r[ t - 1, 1, a ] )
                yBias = ( homePatchY - r[ t - 1, 2, a ] )

                ## random walk steps
                r[ t, 1, a ] = r[ t - 1, 1, a ] + rand( Normal( xBias * drift, walk ) )
                r[ t, 2, a ] = r[ t - 1, 2, a ] + rand( Normal( yBias * drift, walk ) )

            end

            ## if fleeing
            if r[ t - 1, 3, a ] == 1

                ## random walk steps
                r[ t, 1, a ] = r[ t - 1, 1, a ] + rand( Normal( 0, run ) )
                r[ t, 2, a ] = r[ t - 1, 2, a ] + rand( Normal( 0, run ) )

            end

        end

        ## -
        ## Here starts the information updating section
        ## -

        ## extract current coordinates
        C = hcat( r[ 2, 1, : ], r[ 2, 2, : ] )

        ## calculate distances between agents
        D = pairwise( Euclidean(), C; dims = 1 )

        ## threshold this matrix
        D[ D .> sensory ] .= 0

        ## convert to 1 / D ^ 2
        D[ D .> 0 ] = 1 ./ D[ D .> 0 ] .^ 2

        ## -
        ## trigger false alarm
        ## -
        if t == trigger

            ## switch state
            r[ t, 3, 1 ] = 1

        end

    end
    
    ## return results
    return r
    
    end;

In [41]:
scaredyFish()

100×6×10000 Array{Float64,3}:
[:, :, 1] =
 7.01005  28.1136  0.0  1.0  7.0  28.0
 6.91576  28.1527  0.0  1.0  7.0  28.0
 6.90993  28.0697  0.0  1.0  7.0  28.0
 6.9148   28.0062  0.0  1.0  7.0  28.0
 6.88884  27.9938  1.0  1.0  7.0  28.0
 6.88116  27.9106  0.0  1.0  7.0  28.0
 6.8833   27.9139  0.0  1.0  7.0  28.0
 6.85566  27.9346  0.0  1.0  7.0  28.0
 6.92002  28.002   0.0  1.0  7.0  28.0
 6.97818  27.9961  0.0  1.0  7.0  28.0
 6.91918  27.9499  0.0  1.0  7.0  28.0
 6.93647  27.9139  0.0  1.0  7.0  28.0
 6.87305  27.9181  0.0  1.0  7.0  28.0
 ⋮                                 ⋮
 7.2362   27.9223  0.0  1.0  7.0  28.0
 7.30274  27.8767  0.0  1.0  7.0  28.0
 7.23164  27.8696  0.0  1.0  7.0  28.0
 7.14532  27.9169  0.0  1.0  7.0  28.0
 7.15141  27.8065  0.0  1.0  7.0  28.0
 7.20566  27.8616  0.0  1.0  7.0  28.0
 7.15399  27.8408  0.0  1.0  7.0  28.0
 7.1259   27.9182  0.0  1.0  7.0  28.0
 7.10968  27.9152  0.0  1.0  7.0  28.0
 7.0284   27.9248  0.0  1.0  7.0  28.0
 7.01719  27.9626  0.0  